# Business Intelligence Analysis of Amazons Sales Data

**Table of Contents**

1. **[Introduction](#introduction)**
2. **[Data](#data)**
3. **[Sort](#sort)**

### Introduction
<a name="introduction"></a>

In this project i will be implementing techniques and statistics that I learned in my Business Intelligence skillpath on codecademy:

- **Q1** Top 10 most sold products
- **Q2** Highest Rating products
- **Q3** Most expensive products
- **Q4** Total products in each category
- **Q5** Top 5 Categories by Volume sold and Rough Revenue
- **Q6** Is there any correlation between Price and Ratings
- **Q7** Average price in categories.
- **Q8** Create a more informative csv and excel file

<a name="data"></a>
# Import and Inspect Data

In [11]:
#import the neccesary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [33]:
#import the dataset amazon.csv to az
az = pd.read_excel('wrangle-ing.xlsx')
az['actual_value_dollar'] = az['actual_value_dollar'].round(2)
az['discount_value_dollar'] = az['discount_value_dollar'].round(2)
print(az.head())

   product_id                                       product_name  \
0  B07JW9H4J1  Wayona Nylon Braided USB to Lightning Fast Cha...   
1  B098NS6PVG  Ambrane Unbreakable 60W / 3A Fast Charging 1.5...   
2  B096MSW6CT  Sounce Fast Phone Charging Cable & Data Sync U...   
3  B08HDJ86NZ  boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...   
4  B08CF3B7N1  Portronics Konnect L 1.2M Fast Charging 3A 8 P...   

                                            category discounted_price  \
0  Computers&Accessories|Accessories&Peripherals|...           ‚Çπ399   
1  Computers&Accessories|Accessories&Peripherals|...           ‚Çπ199   
2  Computers&Accessories|Accessories&Peripherals|...           ‚Çπ199   
3  Computers&Accessories|Accessories&Peripherals|...           ‚Çπ329   
4  Computers&Accessories|Accessories&Peripherals|...           ‚Çπ154   

  discounted price formatted  discount_value_dollar actual_price  \
0                        399                   4.93     ‚Çπ1,099   
1               

We can see that the prices are reflected in the file with symbols, after investigation in excel, I saw that the currency recored in the data is in Indian Rupees, without hesitation, I popped into the "get info" button on the csv file and it showed me that the document was last modified 17 January 2023, after looking into global currencies I have chosen to display the Rupee as a value in Dollar (United States). 

I did so by looking at the Rupee around the week of 17 January 2023, and have decided on a solid 81 Rupee to Dollar exchange rate.

I did this within an Excel sheet, it is attached to this repository. 

We can see that the Dollar is being displayed with a chain of decimals despite showing up properly in excel to fix this we will round the dollar columns for this analysis.

In [13]:
#check the info
print(az.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 20 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   product_id                       1465 non-null   object 
 1   product_name                     1465 non-null   object 
 2   category                         1465 non-null   object 
 3   discounted_price                 1465 non-null   object 
 4   discounted price formatted       1465 non-null   object 
 5   discount_value_dollar            1465 non-null   float64
 6   actual_price                     1465 non-null   object 
 7   discount price formatted actual  1465 non-null   object 
 8   actual_value_dollar              1464 non-null   float64
 9   discount_percentage              1465 non-null   float64
 10  rating                           1465 non-null   object 
 11  rating_count                     1463 non-null   object 
 12  about_product       

In [14]:
#check the columns in az
print(az.columns)

Index(['product_id', 'product_name', 'category', 'discounted_price',
       'discounted price formatted', 'discount_value_dollar', 'actual_price',
       'discount price formatted actual', 'actual_value_dollar',
       'discount_percentage', 'rating', 'rating_count', 'about_product',
       'user_id', 'user_name', 'review_id', 'review_title', 'review_content',
       'img_link', 'product_link'],
      dtype='object')


<a name="sort"></a>
# Sort Data

In [15]:
#Q1 top_10 (most sold products)
top_sold = az['product_name'].value_counts().reset_index()
top_sold.columns = ['product_name', 'count']
top_10_sold = top_sold.head(10)

In [16]:
#Q2 highest_ratings
#make rating a float value as it is imported as an object number
az['rating'] = pd.to_numeric(az['rating'], errors='coerce')
sorted_ratings = az.sort_values(by='rating', ascending=False)[['product_id', 'product_name', 'rating']]

In [17]:
#Q3 most_exp products (without discoounts)
sorted_exp = az.sort_values(by='actual_value_dollar', ascending=False)
#only return neccesary columns
sorted_exp = sorted_exp[['product_id', 'product_name', 'actual_value_dollar']]

most_exp_10 = sorted_exp.head(10)

In [18]:
#Q4 total products in each category
category_counts = az.groupby('category')['product_name'].count().sort_values(ascending=False)

In [19]:
#Q5 Top 5 Categories by Volume sold and Rough revenue (rough estimate!!)
#bc this data does not include a 'sales volume' column we will make a rough assumption based on the rating_count, taking into consideration that these values are not by any means a true reflection of the volume of sales.
az['rating_count'] = pd.to_numeric(az['rating_count'], errors='coerce') 
az['estimated_revenue'] = az['rating_count'] * az['actual_value_dollar']

#group by category and sum the rough estimated revenue
category_performance = az.groupby('category')['estimated_revenue'].sum().sort_values(ascending=False)
#create a dataframe
category_performance_df = category_performance.reset_index()

#column names
category_performance_df.columns = ['Category', 'Estimated_Revenue']

#format est_rev as money
category_performance_df['Estimated_Revenue'] = category_performance_df['Estimated_Revenue'].apply(lambda x: "${:,.2f}".format(x))

In [22]:
print(az['actual_value_dollar'].dtype)
print(az['rating'].dtype)

float64
float64


In [32]:
#Q6 Is there any correlation between Price and Ratings
#check the correlation for price and rating
corr_pr = az['actual_value_dollar'].corr(az['rating'])
print(corr_pr)

0.11022305681551425


In [45]:
#Q7 Avg price per category
#compare average prices across categories
compare_prices = az.groupby('category')['actual_value_dollar'].mean().reset_index().rename(columns={'actual_value_dollar': 'average_price'})
compare_prices = compare_prices.sort_values('average_price', ascending=False)

In [46]:
#Q8 creating a more informative excel and csv file
columns = ['product_id', 'product_name', 'category', 'actual_value_dollar', 'discount_percentage', 'rating', 'rating_count', 'user_id', 'product_link']

In [47]:
az_refined = az[columns]

In [48]:
#export to csv
az_refined.to_csv('az_refined.csv', index=False)

In [49]:
#export to excel file
az_refined.to_excel('az_refined.xlsx', index=False)